### Simple Web Service (LinearRegression)

In [1]:
import pickle
import numpy as np

In [2]:
with open('../models/lin_reg.bin', 'rb') as f_in:
    model = pickle.load(f_in)

In [3]:
model

Pipeline(steps=[('dv', DictVectorizer()), ('lr', LinearRegression())])

In [4]:
def predict(features):
    preds = model.predict([features])
    result = np.power(10, preds[0])
    
    return float(result)

In [5]:
input_data = {
    'room_type': 'Entire home/apt',
    'neighbourhood': 'SIXTH WARD',
    'latitude': 42.65222,
    'longitude': -73.76724,
    'minimum_nights': 2,
    'number_of_reviews': 302,
    'reviews_per_month': 2.53,
    'availability_365': 253
    }
result = predict(input_data)
print(result)

96.78442429894857


### Web Service with MLflow (XGBoost)

In [6]:
import mlflow

TRACKING_URL = "http://127.0.0.1:5000"
mlflow.set_tracking_uri(TRACKING_URL)

print(f"Tracking URI: '{mlflow.get_tracking_uri()}'")

Tracking URI: 'http://127.0.0.1:5000'


In [7]:
mlflow.search_experiments()

[<Experiment: artifact_location='file:////home/oliver/Desktop/Workspace/Projects/airbnb-price-prediciton/experiment-tracking/../mlruns_artifacts/1', creation_time=1732358023753, experiment_id='1', last_update_time=1732358023753, lifecycle_stage='active', name='airbnb-experiment', tags={}>,
 <Experiment: artifact_location='file:////home/oliver/Desktop/Workspace/Projects/airbnb-price-prediciton/experiment-tracking/../mlruns_artifacts/0', creation_time=1732357512457, experiment_id='0', last_update_time=1732357512457, lifecycle_stage='active', name='Default', tags={}>]

In [8]:
# loaded_model = mlflow.pyfunc.load_model('s3://mlflow-models-alexey/1/6dd459b11b4e48dc862f4e1019d166f6/artifacts/model/')

In [10]:
# RUN_ID = os.getenv('RUN_ID')
RUN_ID = "7b5744464f544451aee4a9308d1971ad"
logged_model = f'runs:/{RUN_ID}/models'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models
  flavor: mlflow.xgboost
  run_id: 7b5744464f544451aee4a9308d1971ad

In [11]:
import os
import shutil
from mlflow.tracking import MlflowClient

# Initialize the MLflow client
client = MlflowClient()

artifact_path = "preprocessor/preprocessor.b"

# Download the artifact to the current working directory
destination_dir = os.getcwd() # Current working directory
artifact_uri = client.download_artifacts(RUN_ID, artifact_path, destination_dir)

# Rename or move the file to the desired location (removing folder structure)
final_path = os.path.join(destination_dir + '/web-service-mlflow', "preprocessor.b")
if os.path.isfile(artifact_uri):
    shutil.move(artifact_uri, final_path)
    
# Remove the empty preprocessor folder
preprocessor_folder = os.path.join(destination_dir, "preprocessor")
if os.path.isdir(preprocessor_folder):
    os.rmdir(preprocessor_folder)

print(f"Artifact downloaded and saved as: {final_path}")

Artifact downloaded and saved as: /home/oliver/Desktop/Workspace/Projects/airbnb-price-prediciton/deployment/web-service-mlflow/preprocessor.b


In [12]:
with open("web-service-mlflow/preprocessor.b", 'rb') as f_in:
    loaded_dv = pickle.load(f_in)

In [13]:
def predict(features):
    X = loaded_dv.transform(features)
    preds = loaded_model.predict(X)
    result = np.power(10, preds[0])
    
    return float(result)

In [14]:
result = predict(input_data)
print(result)

106.03396081447663
